### Load BERT model for classification using Hugging Face

Example in this tutorial is obtained from various Hugging Face documentation pages

In [3]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification

bert_model_name = 'bert-base-uncased'
distilbert_model_name = 'distilbert-base-uncased'

config = AutoConfig.from_pretrained(distilbert_model_name)
tokenizer = AutoTokenizer.from_pretrained(distilbert_model_name)
model = AutoModelForSequenceClassification.from_config(config)

### Load dataset using datasets library

In [4]:
from datasets import load_dataset
train_ds, test_ds = load_dataset('imdb', split=['train', 'test'])

Reusing dataset imdb (/home/chandra/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)


### Or prepare data from scratch (this is similar to using your custom dataset)

In [4]:
#download dataset
#!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#!tar -xf aclImdb_v1.tar.gz

from pathlib import Path
from sklearn.model_selection import train_test_split

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)
    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

# Taking sample dataset for faster processing
train_sample_size = 1000
eval_sample_size = 100
train_texts, train_labels = train_texts[:train_sample_size], train_labels[:train_sample_size]
test_texts, test_labels = test_texts[:eval_sample_size], test_labels[:eval_sample_size]

# further split train data into train and validation sets
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1)


### Obtain encodings using Tokenizer, i.e. convert text into embeddings used by the BERT model

In [5]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

### Prepare dataset for training and validation

In [6]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

### Train the model using the Trainer class

In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
10,0.661560
20,0.472896
30,0.224990
40,0.072443
50,0.018416
60,0.006484
70,0.003522
80,0.002208
90,0.001618
100,0.001210


TrainOutput(global_step=171, training_loss=0.0859363455521433)

### Evaluate model on evaluate and test datasets

In [12]:
eval_results = trainer.evaluate()
print("Evaluation Results: ", eval_results)

Evaluation Results:  {'eval_loss': 0.00015663828526157886, 'epoch': 3.0, 'total_flos': 555351634944000}


In [13]:
test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Test Results: ", test_results)

Test Results:  {'eval_loss': 0.00015650957357138395, 'epoch': 3.0, 'total_flos': 555351634944000}
